# Sequential writing

In [1]:
import bison
import numpy

Let's initialize an empty bison file in writing mode

In [2]:
f = bison.FILE('test', mode='w')

Let's write some data into our bison file. To do so we simply invoke the 'write' method of the 'FILE' class with an additional string, namely a unique tag, to label the content of the data

In [3]:
T = 96
Nc = 5

f.write('T',T)
f.write('ncorr', Nc)
f.write('desc', 'a string describing the data')

[Bison] : T with 6.67572e-06 MB written at 0.0132388 MB/s
[Bison] : ncorr with 9.53674e-06 MB written at 0.0158416 MB/s
[Bison] : desc with 3.62396e-05 MB written at 0.0661445 MB/s


We imagine to have a function which performs a calculation that takes long time

In [4]:
def test_func(n):
    return numpy.random.rand(96*n).reshape(n,96)

For each instance of the calculation we write to disk the results in the bison FILE

In [5]:
for i in [0,1,2,3,4]:
    dat = test_func(Nc)
    f.write(f'calculation number {i}', dat)

[Bison] : calculation number 0 with 0.00376701 MB written at 4.21446 MB/s
[Bison] : calculation number 1 with 0.00376701 MB written at 6.87554 MB/s
[Bison] : calculation number 2 with 0.00376701 MB written at 12.5596 MB/s
[Bison] : calculation number 3 with 0.00376701 MB written at 7.78325 MB/s
[Bison] : calculation number 4 with 0.00376701 MB written at 8.92151 MB/s


The content of the file can be easily imported by opening a new FILE instance in reading mode

In [6]:
ff = bison.FILE('test', mode='r')
res = ff.read()

[Bison] : Reading file test
[Bison] : File created by mbruno at macthxbruno.local on Mon Nov 23 17:18:16 2020
[Bison] : Read 0.0190115 MB at 12.3227 MB/s


In [7]:
res['calculation number 4']

array([[6.33967703e-01, 3.63432354e-01, 2.80614417e-01, 5.43779745e-02,
        3.73402771e-01, 2.08124015e-01, 5.08996251e-01, 4.40775073e-01,
        3.33737937e-02, 6.73353905e-01, 8.30822956e-01, 7.95972056e-01,
        3.34984619e-01, 6.53557977e-01, 6.53803809e-01, 2.56270819e-01,
        6.76398658e-01, 7.94478387e-01, 1.66250463e-01, 1.26240136e-01,
        1.02366751e-01, 8.07957732e-01, 8.91020340e-01, 5.00832718e-01,
        3.24887865e-01, 2.43927859e-04, 5.82046200e-01, 8.52794782e-01,
        5.11940599e-01, 3.84526514e-01, 4.83450540e-01, 6.19501272e-02,
        4.83419556e-01, 2.84769420e-02, 8.36680462e-01, 1.04168599e-01,
        2.54248914e-01, 3.60191568e-01, 8.47256375e-01, 6.87593278e-01,
        7.06837287e-01, 9.87407757e-01, 4.04321108e-01, 7.70412387e-01,
        1.11479950e-02, 9.95091036e-01, 8.64416788e-01, 2.00133852e-01,
        4.87710604e-01, 5.27340973e-01, 4.83145513e-02, 3.41221204e-01,
        4.25665024e-01, 3.90568077e-01, 7.61826032e-01, 2.586466

At this point the bison FILE is still open, which means that no file has yet been created, but instead all data has been dumped into a temporary directory. Let's check it, we should find a `test.temp` directory

In [8]:
!ls -al | grep test

drwxr-xr-x   4 mbruno  staff    128 Nov 23 17:18 test.temp


Now we can close the file and therefore also eliminate the possibility to further append to it, or we can close the file as is, to a new destination, thus leaving the possibility to append more data in the future.

In [9]:
f.close(dest='test2')
!ls -al | grep test

-rw-r--r--   1 mbruno  staff  19939 Nov 23 17:18 test2


Now that a single file has been created, we can import it also with the standard loading command

In [10]:
res = bison.load('test2')
res.keys()

[Bison] : Reading file test2
[Bison] : File created by mbruno at macthxbruno.local on Mon Nov 23 17:18:16 2020
[Bison] : Read 0.0190153 MB at 20.9004 MB/s


dict_keys(['T', 'ncorr', 'desc', 'calculation number 0', 'calculation number 1', 'calculation number 2', 'calculation number 3', 'calculation number 4'])

In [11]:
# final clen up
!rm -r test test2

rm: test: No such file or directory
